Prompts for chat models are built around messages, instead of just plain text.

> 聊天模型的Prompts围绕Message构建,而不是仅仅平面文本

You can make use of templating by using a MessagePromptTemplate. **You can build a ChatPromptTemplate from one or more MessagePromptTemplates**. You can use ChatPromptTemplate's format_prompt -- this returns a PromptValue, which you can convert to a string or Message object, depending on whether you want to use the formatted value as input to an llm or chat model.

> 你可以通过一个或多个`MessagePromptTemplate`构建`ChatPromptTemplate`, 通过`Message`构建`Prompt`<br>

In [9]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

In [10]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [11]:
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [13]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [16]:
# return langchain.prompts.chat.ChatPromptValue
chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.")

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')])

In [17]:
chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

In [18]:
from langchain.chat_models import ChatOpenAI

In [19]:
llm = ChatOpenAI()

In [26]:
llm.invoke(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming."))

AIMessage(content="J'adore la programmation.")

In [27]:
llm.invoke(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())

AIMessage(content="J'adore programmer.")

1. 错误的: 输入类型不能是 ChatPromptValue(messages=[SystemMessage(content=

In [30]:
llm(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming."))

TypeError: Got unknown type ('messages', [SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')])

2. 正确的 ChatPromptValue.to_messages(), 
```
[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]
```

In [29]:
llm(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())

AIMessage(content="J'adore programmer.")

1. 错误的, 不能是 
```
[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]
```

In [31]:
llm.generate(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())

TypeError: Got unknown type ('content', 'You are a helpful assistant that translates English to French.')

2. 正确的, 应该是 
```
[
[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]
]

```

In [32]:
llm.generate([chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()])

LLMResult(generations=[[ChatGeneration(text="J'adore programmer.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="J'adore programmer."))]], llm_output={'token_usage': {'prompt_tokens': 26, 'completion_tokens': 6, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('dd361d63-4eb5-4060-937d-531c46d5d1d6'))])

> 看这里`04.03.02.00.LangModel_chat_models_inerface.ipynb`, generate 是批量接口 因此是   `[  List[Message], List[Message]  ]`